# Neural Network

In [2]:
import numpy as np
import pickle
import gzip


def load_data():
    """
    MNIST comprises of three variables ->
    Training Data:     50,000 entries
    Validation Data:   10,000 entries
    Test Data:         10,000 entries

    One Entry of Input = 28 * 28 = 784
    One Entry of Output = 0 - 9 {integer}

    training_data   =  (inputs, outputs)     - Tuple
    inputs.shape    =  (50,000, 784)         - Numpy Array
    outputs.shape   =  (50,000,)             - Numpy Array
    """
    f = gzip.open('mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = pickle.load(
        f, encoding='iso-8859-1')
    f.close()
    return (training_data, validation_data, test_data)


def load_data_wrapper():
    """
    It Returns Training Data, Validation Data, Test Data

    Training Data is a list of 50,000 entries.
    Each element of this list is a tuple in the form (Input, Output)
    Shape of 1 Input : (784,1)
    Shape of 1 Output : (10,1)

    Validation Data is a list of 10,000 entries.
    Each element of this list is a tuple in the form (Input, Output)
    Shape of 1 Input : (784,1)
    Shape of 1 Output : Integer

    Test Data is a list of 10,000 entries.
    Each element of this list is a tuple in the form (Input, Output)
    Shape of 1 Input : (784,1)
    Shape of 1 Output : Integer
    """
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = list(zip(training_inputs, training_results))
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = list(zip(validation_inputs, va_d[1]))
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = list(zip(test_inputs, te_d[1]))
    return (training_data, validation_data, test_data)


def vectorized_result(j):
    """
    Return a 10 bit vectorized result of a number.
    """
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

In [3]:
import numpy as np
import random

In [4]:
### Mathematical Functions
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def sigmoid_prime(z):
    return sigmoid(z) * (1 - sigmoid(z))

In [5]:
class Network:
    
    def __init__(self, sizes):
        
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]
        
    def feedforward(self, a):
        """Returns the output value."""
        for w, b in zip(self.weights, self.biases):
            z = np.dot(w, a) + b
            a = sigmoid(z)
        return a
    
    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data = None):
        if test_data:
            n_test = len(test_data)
        n = len(training_data)
        for epoch in range(epochs):
            random.shuffle(training_data)
            mini_batches = []
            for k in range(0, n, mini_batch_size):
                mini_batches.append(training_data[k : k + mini_batch_size])
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print('Epoch {0}: {1} / {2}'.
                     format(epoch, self.evaluate(test_data), n_test))
            else:
                print('Epoch {0} complete.'.format(epoch))
    
    def update_mini_batch(self, mini_batch, eta):
        sum_delta_b = [np.zeros(b.shape) for b in self.biases]
        sum_delta_w = [np.zeros(w.shape) for w in self.weights]
        
        for x, y in mini_batch:
            delta_b, delta_w = self.backprop(x, y)
            sum_delta_b = [sdb + db 
                           for sdb, db in zip(sum_delta_b, delta_b)]
            sum_delta_w = [sdw + dw
                           for sdw, dw in zip(sum_delta_w, delta_w)]
            
        m = len(mini_batch)
        self.biases = [b - (eta/m) * sdb
                       for b, sdb in zip(self.biases, sum_delta_b)]
        self.weights = [w - (eta/m) * sdw
                        for w, sdw in zip(self.weights, sum_delta_w)]
    
                      
    def backprop(self, x, y):
        delta_b = [np.zeros(b.shape) for b in self.biases]
        delta_w = [np.zeros(w.shape) for w in self.weights]
        
        # forward pass
        activation = x
        activations = [x]
        zs = []
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation) + b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
            
        # backward pass
        delta = (activations[-1] - y) * sigmoid_prime(zs[-1])
        delta_b[-1] = delta
        delta_w[-1] = np.dot(delta, activations[-2].transpose())
        for l in range(2, self.num_layers):
            z = zs[-l]
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sigmoid_prime(z)
            delta_b[-l] = delta
            delta_w[-l] = np.dot(delta, activations[-l-1].transpose())
            
        return (delta_b, delta_w)
        
    def evaluate(self, test_data):
        result = 0
        for x, y in test_data:
            y_hat = np.argmax(self.feedforward(x))
            if (y_hat == y):
                result += 1
        return result        

In [9]:
net = Network([784, 100, 60, 10])

In [10]:
training_data, validation_data, test_data = load_data_wrapper()

In [11]:
net.SGD(training_data, 10, 30, 3.0, test_data = test_data)

Epoch 0: 8283 / 10000
Epoch 1: 8455 / 10000
Epoch 2: 8515 / 10000
Epoch 3: 8580 / 10000
Epoch 4: 8628 / 10000
Epoch 5: 8632 / 10000
Epoch 6: 8673 / 10000
Epoch 7: 8652 / 10000
Epoch 8: 8670 / 10000
Epoch 9: 8701 / 10000


In [13]:
poornet = Network([784, 10])

In [14]:
poornet.SGD(training_data, 10, 32, 3.0, test_data = test_data)

Epoch 0: 5280 / 10000
Epoch 1: 5560 / 10000
Epoch 2: 5614 / 10000
Epoch 3: 5656 / 10000
Epoch 4: 6032 / 10000
Epoch 5: 6389 / 10000
Epoch 6: 6429 / 10000
Epoch 7: 6440 / 10000
Epoch 8: 7232 / 10000
Epoch 9: 7268 / 10000
